In [10]:
!pip install -q git+https://github.com/openai/whisper.git
!pip install -q gradio language-tool-python


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
!sudo apt-get update
!sudo apt-get install openjdk-17-jdk -y

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease                         
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]      
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]           
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]        
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,151 kB]
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,832 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]     
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu

In [7]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64/"
os.environ["PATH"] += ":/usr/lib/jvm/java-17-openjdk-amd64/bin"


In [8]:
import language_tool_python
tool = language_tool_python.LanguageTool('en-US')


In [15]:
import whisper
import gradio as gr
import language_tool_python

model = whisper.load_model("base")
tool = language_tool_python.LanguageTool('en-US')

def evaluate_grammar(text):
    matches = tool.check(text)
    errors = len(matches)
    words = max(1, len(text.split()))
    score = max(0, 100 - (errors / words) * 100)
    score = round(score, 2)
    return score, errors

def process_audio(audio_file):
    if audio_file is None:
        return "No file uploaded.", None, None
    result = model.transcribe(audio_file)
    text = result["text"]
    score, errors = evaluate_grammar(text)
    return text, f"{score} / 100", f"{errors} grammar issues found"

purple_css = """
body {
    background: linear-gradient(135deg, #4b0082, #7b2cbf, #9d4edd);
    background-attachment: fixed;
}
.gr-blocks {
    color: white !important;
}
.gr-panel, .gr-box {
    background: rgba(255, 255, 255, 0.10) !important;
    border-radius: 16px !important;
    border: 1px solid rgba(255, 255, 255, 0.25) !important;
}
.gr-button {
    background: linear-gradient(135deg, #b026ff, #7b2cff) !important;
    color: white !important;
    font-weight: bold !important;
    border-radius: 12px !important;
    border: none !important;
    box-shadow: 0px 0px 12px rgba(160, 0, 255, 0.6);
}
textarea, .gr-text-input {
    background: rgba(255,255,255,0.15) !important;
    color: white !important;
    border-radius: 12px !important;
    border: 1px solid rgba(255,255,255,0.3) !important;
}
#title-card {
    background: rgba(255, 255, 255, 0.18);
    padding: 20px;
    border-radius: 16px;
    color: white;
    border: 1px solid rgba(255,255,255,0.25);
    text-align: center;
}
"""

with gr.Blocks(css=purple_css) as demo:
    gr.Markdown("<div id='title-card'><h1>🎙️Aditya Grammar Assessment Engine (Voice Sample Upload)</h1></div>")
    gr.Markdown("### 🎤 Upload your audio file (MP3/WAV)")
    audio_input = gr.Audio(type="filepath", label="Upload voice sample")
    btn = gr.Button("Analyze Grammar")
    text_output = gr.Textbox(label="Transcription")
    score_output = gr.Textbox(label="Grammar Score")
    error_output = gr.Textbox(label="Errors Found")
    btn.click(process_audio, inputs=audio_input, outputs=[text_output, score_output, error_output])

demo.launch()


* Running on local URL:  http://127.0.0.1:7866
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://4d26469920d937c1a9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
